In [1]:
# we got the data.csv having fields Charity number, Charity name, Status, Income, Reporting, link of the respective charity
# in step 2 we need to get go into the each of the link in the csv last column
# click on the contact details and we navigate to next page where we get the data of address, phone number, link of website

import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
import pandas as pd
data=pd.read_csv("data.csv",index_col=False)
data.head()

In [12]:
def fetch_last_column_pandas(csv_file_path):
    df = pd.read_csv(csv_file_path)
    return df.iloc[:, -1].tolist()

csv_file = "data.csv"
last_column_data = fetch_last_column_pandas(csv_file)
print(last_column_data)

['https://register-of-charities.charitycommission.gov.uk/charity-search/-/charity-details/205533', 'https://register-of-charities.charitycommission.gov.uk/charity-search/-/charity-details/210183', 'https://register-of-charities.charitycommission.gov.uk/charity-search/-/charity-details/3961959', nan]


In [ ]:
extracted_data = []
driver = webdriver.Chrome()

for link in last_column_data:
    driver.get(link)
    try:
        # Close cookie consent banner if present
        try:
            cross_button = driver.find_element(By.ID, "_com_placecube_cookieconsent_web_portlet_CookieConsentPortlet_cookie-consent-close")
            cross_button.click()
        except:
            pass  # Ignore if cookie consent banner is not present
        
        # Clicking on the contact link to navigate to the details page
        contact_link = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//a[contains(text(), 'Contact')]")))
        contact_link.click()

        # Waiting until an element on the page is visible
        WebDriverWait(driver, 50).until(EC.visibility_of_element_located((By.XPATH, "//dt[contains(text(), 'Address:')]/following-sibling::dd")))

        # Extract address
        address_element = driver.find_element(By.XPATH, "//dt[contains(text(), 'Address:')]/following-sibling::dd[@class='govuk-summary-list__value']")
        address_data = address_element.text.strip().replace('\n', ', ')
        print("Address:", address_data)

        # Extract webpage link
        webpage_link = driver.find_element(By.ID, "charity-contact-website-link")
        webpage_url = webpage_link.get_attribute("href")
        print("Webpage URL:", webpage_url)
        
        # Extract phone number
        phone_element = driver.find_element(By.XPATH, "//dt[contains(text(), 'Phone:')]/following-sibling::dd[@class='govuk-summary-list__value']")
        phone_number = phone_element.text.strip()
        print("Phone Number:", phone_number)
    
    except Exception as e:
        print("Error:", e)

    extracted_data.append([address_data, webpage_url, phone_number])

driver.quit()

In [34]:
import csv
with open('data2.csv', mode='a', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Address', 'Webpage URL', 'Phone Number'])
    writer.writerows(extracted_data)

print("Data written to data2.csv file.")

Data written to data2.csv file.
